Exercise from Think Stats, 2nd Edition (thinkstats2.com)<br>
Allen Downey

In [9]:
import first
import numpy as np
live, firsts, others = first.MakeFrames()
df = live[live.prglngth>30]

The following are the only variables I found that have a statistically significant effect on pregnancy length.

In [2]:
import statsmodels.formula.api as smf
model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1', data=df)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     34.28
Date:                Sun, 31 Jul 2016   Prob (F-statistic):           5.09e-22
Time:                        21:44:47   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8880   BIC:                         3.653e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                38.7617      0.039   1006.410      0.000        38.686    38.837
birthord == 1[T.True]     0.1015      0.040      2.528      0.011         0.023     0.180
race == 2[T.True]         0.1390      0.042      3.311      0.001         0.057     0.221
nbrnaliv > 1[T.True]     -1.4944      0.164     -9.086      0.000        -1.817    -1.172
==============================================================================
Omnibus:                     1587.470   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6160.751
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         10.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Let's go mining for variables that predict sex ratio.

In [3]:
import regression
join = regression.JoinFemResp(live)

In [10]:
def GoMining(df):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    df['boy'] = (df.babysex==1).astype(int)
    print df.shape
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula='boy ~ agepreg + ' + name
            model = smf.logit(formula, data=df)
            nobs = len(model.endog)
            if nobs < len(df)/2:
                continue

            results = model.fit()
        except:
            continue

        variables.append((results.prsquared, name))

    return variables

variables = GoMining(join)
print variables

(9148, 3332)
Optimization terminated successfully.
         Current function value: 0.687464
         Iterations 4
[(0.008046634476674086, 'totalwgt_lb')]


Here are the 30 variables that yield the highest pseudo-$R^2$ value.

In [5]:
regression.MiningReport(variables)

totalwgt_lb 0.00804663447667


Eliminating variables that are not known during pregnancy and others that are fishy for various reasons, here's the best model I could find:

In [11]:
formula='boy ~ agepreg + fmarout5==5 + infever==1'
model = smf.logit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 0.691983
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    boy   No. Observations:                 9148
Model:                          Logit   Df Residuals:                     9144
Method:                           MLE   Df Model:                            3
Date:                Sun, 31 Jul 2016   Pseudo R-squ.:                0.001525
Time:                        22:08:03   Log-Likelihood:                -6330.3
converged:                       True   LL-Null:                       -6339.9
                                        LLR p-value:                 0.0002326
=========================================================================================
                            coef    std err          z      P>|z|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------------
Intercept                -0.1863      0.116     -1.606      0.108        -0.414     0.041
fmarout5 == 5[T.True]     0.1493      0.048      3.087      0.002         0.054     0.244
infever == 1[T.True]      0.2096      0.063      3.304      0.001         0.085     0.334
agepreg                   0.0053      0.004      1.252      0.211        -0.003     0.014
=========================================================================================
"""

Now let's build a model to predict the respondent's parity (number of babies born alive).  I used a nonlinear model of age.  The age3 term is probably overkill, but I thought I would show that it might not be a crazy choice.  It doesn't have much effect on the predictions.

In [12]:
join.numbabes.replace([97], np.nan, inplace=True)
join['age2'] = join.age_r**2
join['age3'] = join.age_r**3

In [13]:
formula='numbabes ~ age_r + age2 + age3 + C(race) + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.677637
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 9148
Model:                        Poisson   Df Residuals:                     9140
Method:                           MLE   Df Model:                            7
Date:                Sun, 31 Jul 2016   Pseudo R-squ.:                 0.03665
Time:                        22:08:04   Log-Likelihood:                -15347.
converged:                       True   LL-Null:                       -15931.
                                        LLR p-value:                6.721e-248
================================================================================
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept       -3.2731      0.719     -4.551      0.000        -4.683    -1.863
C(race)[T.2]    -0.1403      0.014     -9.683      0.000        -0.169    -0.112
C(race)[T.3]    -0.0959      0.024     -3.957      0.000        -0.143    -0.048
age_r            0.3744      0.069      5.433      0.000         0.239     0.510
age2            -0.0090      0.002     -4.158      0.000        -0.013    -0.005
age3           7.12e-05    2.2e-05      3.237      0.001      2.81e-05     0.000
totincr         -0.0185      0.002     -9.858      0.000        -0.022    -0.015
educat          -0.0463      0.003    -15.988      0.000        -0.052    -0.041
================================================================================
"""

Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000

In [14]:
import pandas
columns = ['age_r', 'age2', 'age3', 'race', 'totincr', 'educat']
new = pandas.DataFrame([[35, 35**2, 35**3, 1, 14, 16]], columns=columns)
results.predict(new)

array([ 2.47393019])

To predict marital status, here is the best model I found:

In [15]:
formula='rmarital ~ age_r + age2 + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

/home/ramon/anaconda2/lib/python2.7/site-packages/statsmodels/discrete/discrete_model.py:130: FutureWarning: icol(i) is deprecated. Please use .iloc[:,i]
  endog_dummies = get_dummies(endog.icol(0))


Optimization terminated successfully.
         Current function value: 1.092083
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 9148
Model:                        MNLogit   Df Residuals:                     9113
Method:                           MLE   Df Model:                           30
Date:                Sun, 31 Jul 2016   Pseudo R-squ.:                  0.1661
Time:                        22:08:04   Log-Likelihood:                -9990.4
converged:                       True   LL-Null:                       -11981.
                                        LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept        8.9153      0.792     11.251      0.000         7.362    10.468
C(race)[T.2]    -0.9260      0.087    -10.705      0.000        -1.096    -0.756
C(race)[T.3]    -0.6335      0.133     -4.747      0.000        -0.895    -0.372
age_r           -0.3567      0.050     -7.132      0.000        -0.455    -0.259
age2             0.0047      0.001      6.054      0.000         0.003     0.006
totincr         -0.1301      0.011    -11.475      0.000        -0.152    -0.108
educat          -0.1940      0.018    -10.534      0.000        -0.230    -0.158
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept        2.9927      2.970      1.007      0.314        -2.829     8.815
C(race)[T.2]    -0.3963      0.235     -1.685      0.092        -0.857     0.065
C(race)[T.3]     0.0650      0.336      0.194      0.846        -0.593     0.723
age_r           -0.3141      0.174     -1.806      0.071        -0.655     0.027
age2             0.0064      0.003      2.532      0.011         0.001     0.011
totincr         -0.3217      0.032    -10.135      0.000        -0.384    -0.259
educat          -0.1093      0.048     -2.266      0.023        -0.204    -0.015
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept       -3.6475      1.193     -3.059      0.002        -5.985    -1.310
C(race)[T.2]    -0.3303      0.091     -3.641      0.000        -0.508    -0.153
C(race)[T.3]    -0.8227      0.170     -4.853      0.000        -1.155    -0.490
age_r            0.1238      0.070      1.763      0.078        -0.014     0.261
age2            -0.0008      0.001     -0.814      0.416        -0.003     0.001
totincr         -0.2288      0.011    -20.058      0.000        -0.251    -0.206
educat           0.0661      0.016      4.015      0.000         0.034     0.098
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
Intercept       -2.3978      1.269     -1.889      0.059        -4.886     0.090
C(race)[T.2]    -1.0493      0.101    -10.366      0.000        -1.248    -0.851
C(race)[T.3]    -0.6065      0.154     -3.937      0.000        -0.908    -0.305
age_r            0.2084      0.077      2.699      0.007         0.057     0.360
age2            -0.0030      0.001     -2.619      0.009        -0.005    -0.001
totincr         -0.2900      0.014    -20.314      0.000        -0.318    -0.262
educat          -0.0176      0.021     -0.835      0.404        -0.059     0.024
-

And here is the prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [16]:
columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
new = pandas.DataFrame([[25, 25**2, 2, 11, 12]], columns=columns)
results.predict(new)

array([[ 0.74568869,  0.12829132,  0.0016241 ,  0.03280099,  0.02188668,
         0.06970823]])

So this person has a 75% chance of being currently married, a 13% chance of being "not married but living with opposite sex partner", etc.